In [1]:
!pip install requests

In [2]:
import pandas as pd
import numpy as np
import requests
import json

In [3]:
# read your 1+ StreamingHistory files (depending on how extensive your streaming history is) into pandas dataframes
df_stream0 = pd.read_json("C:/Spotify Account Data/StreamingHistory_music_0.json")
df_stream1 = pd.read_json("C:/Spotify Account Data/StreamingHistory_music_1.json")

# merge streaming dataframes
df_stream = pd.concat([df_stream0, df_stream1])

# create a 'UniqueID' for each song by combining the fields 'artistName' and 'trackName'
df_stream['UniqueID'] = df_stream['artistName'] + ":" + df_stream['trackName']

df_stream.head()

,endTime,artistName,trackName,msPlayed,UniqueID
0,2023-02-06 02:48,Justine Skye,Collide (Sped Up Remix),10880,Justine Skye:Collide (Sped Up Remix)
1,2023-02-06 02:51,Reality Club,Anything You Want,7003,Reality Club:Anything You Want
2,2023-02-06 02:51,Rich Brian,VIVID,682,Rich Brian:VIVID
3,2023-02-06 02:51,SZA,Kill Bill,153946,SZA:Kill Bill
4,2023-02-06 02:55,Rich Brian,Drive Safe,240118,Rich Brian:Drive Safe


In [19]:
# read your edited Library json file into a pandas dataframe
df_library = pd.read_json("C:/Spotify Account Data/YourLibrary.json")

print(df_library.head())

df_tracks=pd.json_normalize(df_library['tracks'])

df_library['artist'] = df_tracks['artist']

df_library['album'] = df_tracks['album']

# add UniqueID column (same as above)
df_library['UniqueID'] = df_tracks['artist'] + ":" + df_tracks['track']

df_library['uri'] = df_tracks['uri']

# add column with track URI stripped of 'spotify:track:'
new = df_tracks["uri"].str.split(":", expand = True)
df_library['track_uri'] = new[2]

df_library.head()

                                              tracks
0  {'artist': 'The Shirelles', 'album': 'Will You...
1  {'artist': 'Noah Kahan', 'album': 'Hurt Somebo...
2  {'artist': 'Jasmine Thompson', 'album': 'Bundl...
3  {'artist': 'Victor Lundberg', 'album': 'It Wil...
4  {'artist': 'Ella Henderson', 'album': 'Ghost',...


,tracks,artist,album,UniqueID,uri,track_uri
0,"{'artist': 'The Shirelles', 'album': 'Will You...",The Shirelles,Will You Still Love Me Tomorrow,The Shirelles:Will You Still Love Me Tomorrow,spotify:track:33AeObqTI5RW6IhHmIyxVT,33AeObqTI5RW6IhHmIyxVT
1,"{'artist': 'Noah Kahan', 'album': 'Hurt Somebo...",Noah Kahan,Hurt Somebody,Noah Kahan:Passenger,spotify:track:6wkVmLTBedKaAoeSj4xP6J,6wkVmLTBedKaAoeSj4xP6J
2,"{'artist': 'Jasmine Thompson', 'album': 'Bundl...",Jasmine Thompson,Bundle of Tantrums,Jasmine Thompson:Let Her Go,spotify:track:5fa6CrQwHwthEoCvTtd5Ig,5fa6CrQwHwthEoCvTtd5Ig
3,"{'artist': 'Victor Lundberg', 'album': 'It Wil...",Victor Lundberg,It Will Be Fine In The End,Victor Lundberg:It Will Be Fine in the End,spotify:track:3ZTNBmBsSTWZ9ANjI5q4Mz,3ZTNBmBsSTWZ9ANjI5q4Mz
4,"{'artist': 'Ella Henderson', 'album': 'Ghost',...",Ella Henderson,Ghost,Ella Henderson:Ghost,spotify:track:2FiceoWDJ67rrmb5tGBpgE,2FiceoWDJ67rrmb5tGBpgE


In [20]:
# create final dict as a copy df_stream
df_tableau = df_stream.copy()

# add column checking if streamed song is in library
# not used in this project but could be helpful for cool visualizations
df_tableau['In Library'] = np.where(df_tableau['UniqueID'].isin(df_library['UniqueID'].tolist()),1,0)

# left join with df_library on UniqueID to bring in album and track_uri
df_tableau = pd.merge(df_tableau, df_library[['album','UniqueID','track_uri']],how='left',on=['UniqueID'])

df_tableau.head()

,endTime,artistName,trackName,msPlayed,UniqueID,In Library,album,track_uri
0,2023-02-06 02:48,Justine Skye,Collide (Sped Up Remix),10880,Justine Skye:Collide (Sped Up Remix),1,Collide (Remix Pack),1r6qzxz77Tvbrvqd1Q1MIN
1,2023-02-06 02:51,Reality Club,Anything You Want,7003,Reality Club:Anything You Want,1,Anything You Want,2QB8FwOszur18Ai7t2XnNi
2,2023-02-06 02:51,Rich Brian,VIVID,682,Rich Brian:VIVID,1,VIVID,5XB4g7ZqHcn84DFJXwBS1i
3,2023-02-06 02:51,SZA,Kill Bill,153946,SZA:Kill Bill,1,SOS,1Qrg8KqiBpW07V7PNxwwwL
4,2023-02-06 02:55,Rich Brian,Drive Safe,240118,Rich Brian:Drive Safe,1,The Sailor,6eDfe957aV4sqGnDhlSikJ


In [5]:
# save your IDs from new project in Spotify Developer Dashboard
CLIENT_ID = '2a97a27b99244cf59e842bb8144e35b4'
CLIENT_SECRET = '25de8c894fa34191aae6e5343e2e86f9'

In [6]:
# generate access token

# authentication URL
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [7]:
# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [8]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [9]:
# create blank dictionary to store track URI, artist URI, and genres
dict_genre = {}

# convert track_uri column to an iterable list
track_uris = df_library['track_uri'].to_list()

# loop through track URIs and pull artist URI using the API,
# then use artist URI to pull genres associated with that artist
# store all these in a dictionary
for t_uri in track_uris:
    
    dict_genre[t_uri] = {'artist_uri': "", "genres":[]}
    
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    a_uri = r['artists'][0]['uri'].split(':')[2]
    dict_genre[t_uri]['artist_uri'] = a_uri
    
    s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)
    s = s.json()
    dict_genre[t_uri]['genres'] = s['genres']

In [13]:
# convert dictionary into dataframe with track_uri as the first column
df_genre = pd.DataFrame.from_dict(dict_genre, orient='index')
df_genre.insert(0, 'track_uri', df_genre.index)
df_genre.reset_index(inplace=True, drop=True)

df_genre.head()

,track_uri,artist_uri,genres
0,33AeObqTI5RW6IhHmIyxVT,0x83OBqixqdCHnStP5VMcn,"[classic girl group, classic soul, doo-wop, mo..."
1,6wkVmLTBedKaAoeSj4xP6J,2RQXRUsr4IW1f3mKyKsy4B,[pov: indie]
2,5fa6CrQwHwthEoCvTtd5Ig,2TL8gYTNgD6nXkyuUdDrMg,[viral pop]
3,3ZTNBmBsSTWZ9ANjI5q4Mz,3aPt0ITiKD1r1GXV3RvOUh,[]
4,2FiceoWDJ67rrmb5tGBpgE,7nDsS0l5ZAzMedVRKPP8F1,"[dance pop, pop dance, talent show, uk pop]"


In [12]:
df_genre_expanded = df_genre.explode('genres')
df_genre_expanded.head()

,track_uri,artist_uri,genres
0,33AeObqTI5RW6IhHmIyxVT,0x83OBqixqdCHnStP5VMcn,classic girl group
0,33AeObqTI5RW6IhHmIyxVT,0x83OBqixqdCHnStP5VMcn,classic soul
0,33AeObqTI5RW6IhHmIyxVT,0x83OBqixqdCHnStP5VMcn,doo-wop
0,33AeObqTI5RW6IhHmIyxVT,0x83OBqixqdCHnStP5VMcn,motown
0,33AeObqTI5RW6IhHmIyxVT,0x83OBqixqdCHnStP5VMcn,rock-and-roll


In [21]:
# save df_tableau and df_genre_expanded as csv files that we can load into Tableau
df_tableau.to_csv('MySpotifyDataTable.csv')
df_genre_expanded.to_csv('GenresExpandedTable.csv')

print('done')

done
